# List of My Movies

In [1]:
## Basic stuff
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))

## Python Version
import sys
print("Python: {0}".format(sys.version))

from combine import combine
from fsUtils import moveFile


import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

Python: 3.6.6 |Anaconda custom (64-bit)| (default, Jun 28 2018, 11:07:29) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2019-02-14 16:17:06.220888


In [20]:
import re
from time import sleep
from collections import OrderedDict
from timeUtils import clock, elapsed
from ioUtils import saveFile, getFile
from fsUtils import setDir, isDir, mkDir, setFile, isFile, setSubFile, moveFile
from fileUtils import getBaseFilename, getDirname, getFileBasics
from searchUtils import findSubPatternExt, findPatternExt, findExt, findNearest
from strUtils import convertCurrency
from webUtils import getWebData, getHTML
from numpy import repeat
from movieDB import movieDB
from os import getcwd
from os.path import join
from glob import glob
import operator
from movieRenames import manualRenames

##############################################################################################################################
# Combine Movies
##############################################################################################################################
class mymovies(movieDB):
    def __init__(self, basedir=None):
        self.name = "mymovies"
        movieDB.__init__(self, dbdir=self.name)
        self.combine = None
        
    def setCombinedMovies(self, combine):
        self.combine = combine
        
    def getCombinedMovies(self, debug=False):        
        savename = setFile(self.combine.getResultsDir(), "movies.json")
        if not isFile(savename):
            raise ValueErrro("Cannot access {0}".format(savename))
        combinedMovies = getFile(savename)
        if debug:
            print("Found {0} combined movies".format(len(combinedMovies)))
        return combinedMovies
    
    def setMissingMovies(self, missing):
        self.missingMovies = missing
        
    def getMissingMovies(self):
        return self.missingMovies
        
    def getMyMovies(self, debug=False): 
        savename = setFile(self.getDataDir(), "mymovies.json")
        if not isFile(savename):
            raise ValueErrro("Cannot access {0}".format(savename))
        mine = getFile(savename)
        if debug:
            print("Found {0} my movies".format(len(mine)))
        return mine
        
    def findMyMovies(self, debug=False):
        movies = glob("/Volumes/*/Movies/*")
        mine   = dict(zip([getBaseFilename(x) for x in movies], movies))
        print("Found {0} movies on my disks".format(len(movies)))
        savename = setFile(self.getDataDir(), "mymovies.json")
        saveFile(idata=mine, ifile=savename, debug=True)
        
        
    def searchMyMovies(self, movie, year, cutoff=0.8):
        moviesMine = self.getMyMovies()
        key = "{0} [{1}]".format(movie, year)
        result     = findNearest(key, moviesMine.keys(), 3, 0.95)
        if len(result) > 0:
            print("{0}  --->  {1}".format(movie, result))
            location = moviesMine[result[0]]
            print("moveFile(src=\"{0}\", dst=\"{1}\", debug=True)\n".format(location, location))
            return
        else:
            key = "{0} [{1}]".format(movie, year)
            result = findNearest(key, moviesMine.keys(), 3, cutoff)
            if len(result) > 0:
                print("{0}  --->  {1}".format(movie, result))
                location = moviesMine[result[0]]
                print("moveFile(src=\"{0}\", dst=\"{1}\", debug=True)\n".format(location, location))
                return
        print("No match found")
        return
    
    
    def suggestRenames(self):

        movies = glob("/Volumes/Download/MoviesFinished/*.*")
        for movie in movies:
            fileInfo = getFileBasics(movie)
            name = fileInfo[1]
                        
            ps=["1080", "720", "480"]
            for p in ps:
                name = name.replace("{0}p".format(p), "")

            xs=['264']
            for x in xs:
                name = name.replace("x{0}".format(x), "")
            
            ts=["BluRay", "BRrip", "WEBRip", "HDTVRip", "GAZ", "BrRip"]
            for t in ts:
                name = name.replace("{0}".format(t), "")
                
            vs=["-[YTS.AG]", "-[YTS.AM]", "YIFY", "[YTS.AG]"]
            for v in vs:
                name = name.replace("{0}".format(v), "")
                
            name = name.replace(".", " ")
            name=name.strip()

            dst = join(fileInfo[0], "".join([name, fileInfo[2]]))
            if name != fileInfo[1]:
                moveFile(src=movie, dst=dst, debug=True)


        movies = glob("/Volumes/Download/MoviesFinished/*.*")   
        for movie in movies:
            fileInfo = getFileBasics(movie)            
            name  = fileInfo[1][:-4].strip()
            year  = fileInfo[1][-4:]
            
            try:
                int(year)
                name = "{0} [{1}]".format(name, year)
            except:
                print("Cannot rename {0}".format(movie))
                continue
            
            src = movie
            dst = join(fileInfo[0], "".join([name, fileInfo[2]]))
            moveFile(src=src, dst=dst, debug=True)
            print("")
        
        
    def mergeMovies(self, toFile=False, movie=None, source=None, year=None):
        moviesCombined = self.getCombinedMovies()
        moviesMine     = self.getMyMovies()
        renames        = {}
        filename = setFile(self.getResultsDir(), "mymovies.dat")
        
        
        yearcut = year
        sourcecut = source
        moviecut = movie

        if toFile:
            print("{0: <6}{1: <3}{2: <100}{3: <20}".format("Year", u'\u2713', "Movie", "Source"), file=open(filename, "w"))
        else:
            print("{0: <6}{1: <3}{2: <100}{3: <20}".format("Year", u'\u2713', "Movie", "Source"))

        pYear = None
        yearRenames = {}
        
        
        toGet = {}
        
        for key,name in moviesCombined.items():
            movie   = key[:-7]
            year = key[-5:-1]
            
            if yearcut is not None:
                if int(year) not in [int(yearcut)-1,int(yearcut),int(yearcut)+1]:
                    continue
            if sourcecut is not None:
                if name != sourcecut:
                    continue
            if moviecut is not None:
                if moviecut not in movie:
                    continue

        
            location = moviesMine.get(key)
            if location is None:
                for dYear in [-1,1,-2,2]:
                    dKey = "{0} [{1}]".format(movie, int(year)+dYear)
                    if moviesMine.get(dKey) is not None:
                        location = moviesMine[dKey]
                        break
            if location is None:
                result = findNearest(key, moviesMine.keys(), 1, 0.98)

                if len(result) > 0:
                    location = u'\u2248'
                    myMovie  = moviesMine[result[0]]
                    fileInfo = getFileBasics(myMovie)
                    myMovieYear = fileInfo[1][-5:-1]

                    yearRenames[myMovie] = join(fileInfo[0], "".join([key, fileInfo[2]]))

                    if abs(int(year) - int(myMovieYear)) <= 0:
                        renames[myMovie] = join(fileInfo[0], "".join([key, fileInfo[2]]))
                else:
                    result = findNearest(key, moviesMine.keys(), 1, 0.90)
                    if len(result) > 0:
                        location = u'\u2047'
                        myMovie  = moviesMine[result[0]]
                        fileInfo = getFileBasics(myMovie)
                        myMovieYear = fileInfo[1][-5:-1]

                        yearRenames[myMovie] = join(fileInfo[0], "".join([key, fileInfo[2]]))

                        if abs(int(year) - int(myMovieYear)) <= 0:
                            renames[myMovie] = join(fileInfo[0], "".join([key, fileInfo[2]]))
                    else:
                        location = ""
                        
                        if toGet.get(year) is None:
                            toGet[year] = set()
                        toGet[year].add(movie)
            else:
                location = u'\u2713'
                
            
            #print(pYear,year)
            if pYear is not None and pYear != year:
                print("\n")
                for key,rename in yearRenames.items():
                    if key != rename:
                        print("moveFile(src=\"{0}\", dst=\"{1}\", debug=True)".format(key, rename))                
                
                print("\n")
                yearRenames = {}
            pYear = year
                
            if toFile:
                print("{0: <6}{1: <3}{2: <100}{3: <20}".format(year, location, movie, name), file=open(filename, "a"))
            else:
                print("{0: <6}{1: <3}{2: <100}{3: <20}".format(year, location, movie, name))

                
                
        print("\n")
        for key,rename in renames.items():
            if key != rename:
                print("moveFile(src=\"{0}\", dst=\"{1}\", debug=True)\n".format(key, rename))

        self.setMissingMovies(toGet)
        print("Missing Movies Found in getMissingMovies()")

In [21]:
mine = mymovies()
mine.setCombinedMovies(combine())
_ = clock("Last Run")

Current Time is Thu Feb 14, 2019 16:52:37 for Last Run


In [22]:
mine.suggestRenames()

In [19]:
mine.findMyMovies()
_ = clock("Last Run")

Found 5321 movies on my disks
Saving data to /Users/tgadfort/Documents/code/movies/mymovies/data/mymovies.json
Saved data to /Users/tgadfort/Documents/code/movies/mymovies/data/mymovies.json
  --> This file is 431.0kB.
Current Time is Thu Feb 14, 2019 16:52:24 for Last Run


In [12]:
mine.searchMyMovies("The Nutcracker", 2009, cutoff=0.6)

The Nutcracker  --->  ['The Nutcracker [2010]']
moveFile(src="/Volumes/Download/Movies/The Nutcracker [2010].mp4", dst="/Volumes/Download/Movies/The Nutcracker [2010].mp4", debug=True)



In [23]:
mine.mergeMovies()

Year  ✓  Movie                                                                                               Source              
1915  ✓  The Birth Of A Nation                                                                               Rotten Tomatoes     




1920     The Cabinet Of Dr. Caligari (Das Cabinet des Dr. Caligari)                                          Rotten Tomatoes     




1921  ✓  The Four Horsemen Of The Apocalypse                                                                 Wiki Film           
1921  ✓  The Kid                                                                                             Wiki Film           
1921  ✓  Orphans Of The Storm                                                                                Wiki Film           
1921  ⁇  The Three Musketeers                                                                                Wiki Film           
1921  ✓  The Sheik                                                                

1926     Beau Geste                                                                                          Wiki Film           
1926     The Better Ole                                                                                      Wiki Film           
1926     Flesh And The Devil                                                                                 Wiki Film           
1926     Sparrows                                                                                            Wiki Film           
1926     The Temptress                                                                                       Wiki Film           
1926     The Sea Beast                                                                                       Wiki Film           
1926     My Yankee Senor                                                                                     Wiki Film           




1927  ✓  Metropolis                                                                   

1931     Hell Divers                                                                                         Wiki Film           
1931     Possessed                                                                                           Wiki Film           
1931     Susan Lenox                                                                                         Wiki Film           
1931     Reducing                                                                                            Wiki Film           
1931     Daddy Long Legs                                                                                     Wiki Film           




1932     Freaks                                                                                              Rotten Tomatoes     
1932  ✓  Shanghai Express                                                                                    Ultimate Movie Rankings
1932  ✓  Grand Hotel                                                               

1935     David Copperfield                                                                                   Oscar               
1935  ⁇  Les Misérables                                                                                      Oscar               
1935     A Midsummer Night's Dream                                                                           Oscar               
1935     Captain Bligh                                                                                       Oscar               
1935     Black Fury                                                                                          Oscar               
1935     Franchot Tone                                                                                       Oscar               
1935     Escape Me Never                                                                                     Oscar               
1935     Private Worlds                                                                   

1937     Camille                                                                                             Oscar               
1937     Stella Dallas                                                                                       Oscar               
1937     Dead End                                                                                            Oscar               
1937  ⁇  The Hurricane                                                                                       Oscar               
1937     Topper                                                                                              Oscar               
1937     A Damsel In Distress                                                                                Oscar               
1937     One Hundred Men And A Girl                                                                          Oscar               
1937     Penny Wisdom                                                                     

...

2017     Downsizing                                                                                          Box Office Mojo     
2017     Flatliners                                                                                          Box Office Mojo     
2017  ✓  Detroit                                                                                             Box Office Mojo     
2017  ✓  The Case For Christ                                                                                 Box Office Mojo     
2017     Wish Upon                                                                                           Box Office Mojo     
2017  ✓  It Comes At Night                                                                                   Box Office Mojo     
2017     Born In China                                                                                       Box Office Mojo     
2017  ✓  Megan Leavey                                                                     

2018     Leaning Into The Wind                                                                               Rotten Tomatoes     
2018     Ryuichi Sakamoto: Coda                                                                              Rotten Tomatoes     
2018     Black Panther                                                                                       Rotten Tomatoes     
2018     Mission: Impossible - Fallout                                                                       Rotten Tomatoes     
2018     Spider-Man: Into The Spider-Verse                                                                   Rotten Tomatoes     
2018     The Hate U Give                                                                                     Rotten Tomatoes     
2018     The Rider                                                                                           Rotten Tomatoes     
2018  ✓  They Shall Not Grow Old                                                          

2018     Fayssal Bazzi                                                                                       AACTA               
2018     1%                                                                                                  AACTA               
2018     Ladies In Black                                                                                     AACTA               
2018     Josh McConville                                                                                     AACTA               
2018     Mary Magdalene                                                                                      AACTA               
2018     Avengers: Infinity War                                                                              Box Office Mojo     
2018  ✓  Jurassic World: Fallen Kingdom                                                                      Box Office Mojo     
2018  ✓  Jumanji: Welcome To The Jungle                                                   

2018     The 15:17 To Paris                                                                                  Box Office Mojo     
2018  ✓  Mile 22                                                                                             Box Office Mojo     
2018  ⁇  Under The Sea 3D                                                                                    Box Office Mojo     
2018  ✓  Alpha                                                                                               Box Office Mojo     
2018  ✓  Peppermint                                                                                          Box Office Mojo     
2018     Death Wish                                                                                          Box Office Mojo     
2018  ✓  Geostorm                                                                                            Box Office Mojo     
2018  ✓  The Spy Who Dumped Me                                                            

2018  ⁇  Fahrenheit 11/9                                                                                     Box Office Mojo     
2018     Unbroken: Path To Redemption                                                                        Box Office Mojo     
2018     The Hurricane Heist                                                                                 Box Office Mojo     
2018     Every Day                                                                                           Box Office Mojo     
2018     Just Getting Started                                                                                Box Office Mojo     
2018     Once Upon A Deadpool                                                                                Box Office Mojo     
2018     Death Of A Nation                                                                                   Box Office Mojo     
2018     My Hero Academia: Two Heroes                                                     

In [24]:

moveFile(src="/Volumes/Save/Movies/All The Kings Men [2006].mkv", dst="/Volumes/Save/Movies/All The King's Men [2006].mkv", debug=True)

Moving /Volumes/Save/Movies/All The Kings Men [2006].mkv to /Volumes/Save/Movies/All The King's Men [2006].mkv


In [25]:

moveFile(src="/Volumes/Save/Movies/Chico Y Rita [2011].avi", dst="/Volumes/Save/Movies/Chico & Rita [2011].avi", debug=True)


Moving /Volumes/Save/Movies/Chico Y Rita [2011].avi to /Volumes/Save/Movies/Chico & Rita [2011].avi
